In [1]:
import cv2
import kornia as K
import kornia.feature as KF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

import kornia_moons
from kornia_moons.feature import *
from PIL import Image


In [2]:
im_path =  '/data/s0/udc/dataset/UDC_SIT_visualize/' 
udc_dataset = pd.read_csv("./udc.csv")
udc_dataset.species.value_counts().head(8)


species
udc    4680
Name: count, dtype: int64

In [3]:
udc_list = udc_dataset["individual_id"].values.tolist()
udc_list = list(set(udc_list))

print(len(udc_list))

2340


In [4]:
def load_torch_image(fname):
    img = K.image_to_tensor(cv2.imread(fname), False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img


In [5]:
global MAX_SIZE 
MAX_SIZE = 1000

DRAW = False

In [6]:
torch.cuda.empty_cache()

In [7]:
import torch
import torch.cuda
import torchvision

def load_torch_image(fname, device):
    img: Tensor = K.io.load_image(fname, K.io.ImageLoadType.RGB32)
    img = img[None]
    img = K.geometry.resize(img, (1792, 1280))
    return img.to(device)


def match_and_draw_gpu(im_path, img_in1, img_in2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    img1 = load_torch_image(im_path + img_in1, device)
    img2 = load_torch_image(im_path + img_in2, device)
    
    matcher = KF.LoFTR(pretrained='outdoor')
    matcher = torch.nn.DataParallel(matcher)
    matcher = matcher.to(device)

    input_dict = {
        "image0": K.color.rgb_to_grayscale(img1).to(device), # Convert to grayscale and send to GPU
        "image1": K.color.rgb_to_grayscale(img2).to(device)  # Convert to grayscale and send to GPU
    }

    with torch.no_grad():
        correspondences = matcher(input_dict)

    if DRAW:
        mkpts0 = correspondences['keypoints0'].cpu().numpy()
        mkpts1 = correspondences['keypoints1'].cpu().numpy()

        H, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)
        inliers = inliers > 0

        kornia_moons.feature.draw_LAF_matches(
            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1, -1, 2),
                                        torch.ones(mkpts0.shape[0]).view(1, -1, 1, 1),
                                        torch.ones(mkpts0.shape[0]).view(1, -1, 1)), 

            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1, -1, 2),
                                        torch.ones(mkpts1.shape[0]).view(1, -1, 1, 1),
                                        torch.ones(mkpts1.shape[0]).view(1, -1, 1)), 

            torch.arange(mkpts0.shape[0]).view(-1, 1).repeat(1, 2),
            K.tensor_to_image(img1),
            K.tensor_to_image(img2),
            inliers,
            draw_dict={'inlier_color': (0.2, 1, 0.2),
                    'tentative_color': None,
                    'feature_color': (0.2, 0.5, 1), 'vertical': False}
        )
    
    return correspondences


In [8]:
def calculate_pck(correspondences, ind, pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr, dirname):
    # Keypoint coordinates for last prediction - only for showing structure 
    key_gt = correspondences['keypoints0'].cpu().numpy().T
    key_in = correspondences['keypoints1'].cpu().numpy().T

    ind = int(ind)
    np.savetxt('./'+dirname+'/keypoints_'+str(ind)+'_gt.txt', key_gt, delimiter=',', fmt='%.1f')
    np.savetxt('./'+dirname+'/keypoints_'+str(ind)+'_input.txt', key_in, delimiter=',', fmt='%.6f')

    diff = np.abs(key_gt - key_in)
    diff = np.ndarray.flatten(diff)
    count_0p002, count_0p005, count_0p01, count_0p03, count_0p10 = 0, 0, 0, 0, 0

    for i in range(len(diff)):
        if diff[i] <= MAX_SIZE * 0.002:
            count_0p002 += 1
        if diff[i] <= MAX_SIZE * 0.005:
            count_0p005 += 1    
        if diff[i] <= MAX_SIZE * 0.01:
            count_0p01 += 1
        if diff[i] <= MAX_SIZE * 0.03:
            count_0p03 += 1
        if diff[i] <= MAX_SIZE * 0.10:
            count_0p10 += 1

    pck_0p002, pck_0p005, pck_0p01, pck_0p03, pck_0p10 = count_0p002 / len(diff), count_0p005 / len(diff), count_0p01 / len(diff), count_0p03 / len(diff), count_0p10 / len(diff)
    print(".......Each PCK:", pck_0p002, pck_0p005, pck_0p01, pck_0p03, pck_0p10)

    pck_0p002_arr.append(pck_0p002), pck_0p005_arr.append(pck_0p005), pck_0p01_arr.append(pck_0p01), pck_0p03_arr.append(pck_0p03), pck_0p10_arr.append(pck_0p10)
    print(".......AVG PCK:", np.mean(pck_0p002_arr), np.mean(pck_0p005_arr), np.mean(pck_0p01_arr), np.mean(pck_0p03_arr), np.mean(pck_0p10_arr))

    np.savetxt('./'+dirname+'/pck_0p002.txt', pck_0p002_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p005.txt', pck_0p005_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p01.txt', pck_0p01_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p03.txt', pck_0p03_arr, delimiter=',', fmt='%.4f')
    np.savetxt('./'+dirname+'/pck_0p10.txt', pck_0p10_arr, delimiter=',', fmt='%.4f')

    return pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr

In [9]:
import os

dirname = 'res_udc'
if not os.path.exists(dirname):
    os.makedirs(dirname)

In [10]:
pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr = [], [], [], [], []

for i in udc_list:
    query_str = "individual_id == " + str(i)
    img_to_draw = [file for file in udc_dataset.query(query_str).image]

    udc_1 = img_to_draw[1]
    udc_2 = img_to_draw[0]
    print(f'Matching: {udc_1} to {udc_2}')

    correspondences = match_and_draw_gpu(im_path, udc_2, udc_1)

    pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr = calculate_pck(correspondences, i, pck_0p002_arr, pck_0p005_arr, pck_0p01_arr, pck_0p03_arr, pck_0p10_arr, dirname)

Matching: 1_input.png to 1_GT.png
.......Each PCK: 0.9757579553996492 0.9838386369330995 0.9924830869456277 0.9956777749937359 0.9998120771736407
.......AVG PCK: 0.9757579553996492 0.9838386369330995 0.9924830869456277 0.9956777749937359 0.9998120771736407
Matching: 2_input.png to 2_GT.png
.......Each PCK: 0.9644380010233669 0.9809824322019444 0.9983796691113764 0.9988060719768037 0.99906191369606
.......AVG PCK: 0.970097978211508 0.9824105345675219 0.995431378028502 0.9972419234852699 0.9994369954348503
Matching: 3_input.png to 3_GT.png
.......Each PCK: 0.9545586738403373 0.9715719063545151 0.9907663225243566 0.992729387814454 0.994328922495274
.......AVG PCK: 0.9649182100877844 0.9787976584965197 0.9938763595271203 0.9957377449283312 0.9977343044549917
Matching: 4_input.png to 4_GT.png
.......Each PCK: 0.7478059677676719 0.8784107228338918 0.9928993138662837 0.9954523695548109 0.9969682463698739
.......AVG PCK: 0.9106401495077563 0.9537009245808628 0.9936320981119111 0.99566640108495